## **1. Data Collection**

### **1.1. Get the list of Michelin restaurants**

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from IPython.display import display

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
} # user agent is used to simulate that the http request comes from a real web browser, this prevent the server from blocking requests

def guide_michelin(): # 2037
        links = []
        for i in range(1,101): #100
            link = "https://guide.michelin.com/en/it/restaurants/page/{}".format(i)
            try:
                response = requests.get(link, headers=headers)
            except Exception as e:
                print(f"{e} \n {link}")
                continue
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                section = soup.find('div', class_="row restaurant__list-row js-restaurant__list_items")
                if section:  
                    for a_tag in section.find_all('a', href=True):
                        href = 'https://guide.michelin.com' + a_tag['href']
                        if href not in links and "/restaurant/" in href: 
                            links.append(href)
            else:
                print(f"Failed to retrieve page {i}")    
        return links

url_set = guide_michelin()
print(len(url_set))

1981


In [7]:
with open('links.txt', 'w') as f:
    for url in url_set:
        f.write(url + '\n')

### **1.2. Crawl Michelin restaurant pages**

In [8]:
if not os.path.exists('pages'):
    os.makedirs('pages')

with open('links.txt', 'r') as f:
    urls = f.read().splitlines()

# Create directories and save HTML documents
for index, url in enumerate(urls):
    page_number = index // 20 + 1
    directory = os.path.join('pages', f'page_{page_number}')
    if not os.path.exists(directory):
        os.makedirs(directory)
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            file_path = os.path.join(directory, f'document_{index}.html')
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)
        else:
            print(f"Failed to retrieve {url}")
    except Exception as e:
        print(f"Error fetching {url}: {e}")

print("HTML documents saved successfully.")

HTML documents saved successfully.


In [9]:
dir_paths = [os.path.join('pages', dir) for dir in os.listdir('pages')]
len(dir_paths)

100

### **1.3. Parse downloaded pages**

In [2]:
# Function to extract restaurant details from HTML content
def extract_restaurant_details(content):
    
    # Extract the restaurant name
    name = content.find('h1', class_='data-sheet__title').get_text(strip=True) if content.find('h1', class_='data-sheet__title') else ""
    
    # Extract the first row of basic information
    firstRow = content.find_all("div", class_="data-sheet__block--text")[0].get_text(strip=True)
    #firstRow = content.find("div", class_="data-sheet__block--text").get_text(strip=True)
    firstRow_list = [info.strip() for info in firstRow.split(",")]

    address = " ".join(firstRow_list[:-3]) if len(firstRow_list) > 3 else ""
    city = firstRow_list[-3] if len(firstRow_list) > 2 else ""
    postalCode = firstRow_list[-2] if len(firstRow_list) > 1 else ""
    country = firstRow_list[-1] if firstRow_list else ""

    # Extract the second row of basic information
    secondRow = content.find_all("div", class_="data-sheet__block--text")[1].get_text(strip=True)
    #secondRow = content.find("div", class_="data-sheet__block--text").get_text(strip=True)
    secondRow_list = [info.strip() for info in secondRow.split("·")]

    priceRange = secondRow_list[0] if secondRow_list else ""
    cuisineType = secondRow_list[1] if len(secondRow_list) > 1 else ""

    # Extract the description
    description = content.find("div", class_="data-sheet__description").get_text(strip=True) if content.find("div", class_="data-sheet__description") else ""

    # Extract facilities and services
    facilitiesServices_div = content.find_all("div", class_="col col-12 col-lg-6")
    # facilitiesServices_div = content.find("div", class_="col col-12 col-lg-6")
    facilitiesServices = [li.get_text(strip=True) for li in facilitiesServices_div[0].find_all("li")] if facilitiesServices_div else []
    # facilitiesServices = [li.get_text(strip=True) for li in facilitiesServices_div.find("li")] if facilitiesServices_div else []

    # Extract credit card information
    creditCards_div = content.find("div", class_="restaurant-details__services--info")
    creditCards = [os.path.basename(img["data-src"]).split("-")[0] for img in creditCards_div.find_all("img")] if creditCards_div else []

    # Extract phone number
    phoneNumber = content.find("span", attrs={"x-ms-format-detection": "none"}).get_text(strip=True) if content.find("span", attrs={"x-ms-format-detection": "none"}) else ""

    # Extract website
    website_div = content.find("div", class_="collapse__block-item link-item")
    website = website_div.find("a", class_="link js-dtm-link")["href"] if website_div and website_div.find("a", class_="link js-dtm-link") else ""

    # Return the extracted data as a dictionary
    return {
        "restaurantName": name,
        "address": address,
        "city": city,
        "postalCode": postalCode,
        "country": country,
        "priceRange": priceRange,
        "cuisineType": cuisineType,
        "description": description,
        "facilitiesServices": facilitiesServices,
        "creditCards": creditCards,
        "phoneNumber": phoneNumber,
        "website": website
    }

# Collecting data from all HTML files
#folder_paths = [d for d in os.listdir('pages') if os.path.isdir(d) and d.startswith("page_")]
dir_paths = [os.path.join('pages', dir) for dir in os.listdir('pages')]

data = []
for dir in dir_paths:
    for html_file in os.listdir(dir):
        if html_file.endswith(".html"):
            with open(os.path.join(dir, html_file), "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file, "html.parser")
                restaurant_details = extract_restaurant_details(soup)
                data.append(restaurant_details)

# Create a DataFrame from the data list
df = pd.DataFrame(data)

df.columns = ["restaurantName", "address", "city", "postalCode", "country", "priceRange", "cuisineType", "description", "facilitiesServices", "creditCards", "phoneNumber", "website"]

In [3]:
# Display the DataFrame
display(df)

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Da Annetta,via Fè 25,Capolago,21100,Italy,€€,"Traditional Cuisine, Modern Cuisine","In an 18th Century building, a rustic yet eleg...","[Air conditioning, Car park, Interesting wine ...","[amex, mastercard, visa]",+39 0332 490230,https://www.ristoranteannetta.it/
1,Osaka,corso Giuseppe Garibaldi 68,Milan,20121,Italy,€€€,"Japanese, Traditional Cuisine",Tucked away in a short arcade on the old road ...,"[Air conditioning, Counter dining, Notable sak...","[amex, unionpay, maestrocard, mastercard, visa]",+39 02 2906 0678,https://www.milanoosaka.com
2,Gourmetstube Einhorn,Campo di Trens,Mules,39040,Italy,€€€€,"Creative, International","Occupying a 13C post house, this restaurant is...","[Car park, Garden or park, Interesting wine list]","[dinersclub, maestrocard, mastercard, visa]",+39 0472 771136,https://www.stafler.com/it/hotel-gourmet-alto-...
3,Benso,piazza Cavour 7,Forlì,47121,Italy,€€,"Modern Cuisine, Italian Contemporary",Situated in a small garden in the historic cen...,"[Air conditioning, Terrace, Wheelchair access]","[amex, mastercard, visa]",+39 346 116 7238,https://www.bensofood.com/
4,La Notizia 53,via Caravaggio 53/55,Naples,80126,Italy,€,Pizza,Although situated away from the tourist centre...,[Air conditioning],"[amex, maestrocard, mastercard, visa]",+39 081 714 2155,http://www.pizzarialanotizia.com
...,...,...,...,...,...,...,...,...,...,...,...,...
1978,Foresta,via Litoranea 2,Marina di Pisa,56128,Italy,€€€,"Seafood, Classic Cuisine","Overlooking the Tyrrhenian Sea, all the tables...","[Air conditioning, Great view, Terrace, Wheelc...","[amex, mastercard, visa]",+39 050 35082,https://www.ristoranteforesta.com/
1979,Oberraindlhof,Raindl 49,Madonna di Senales,39020,Italy,€€,"Traditional Cuisine, Regional Cuisine",The family that run this restaurant occupying ...,"[Car park, Great view, Interesting wine list, ...","[amex, maestrocard, mastercard, visa]",+39 0473 679131,https://www.oberraindlhof.com/it/hotel-per-buo...
1980,Antica Trattoria Gianna,via Maggiore 12,Recorfano,26030,Italy,€,"Lombardian, Country cooking","A delightful family trattoria offering simple,...","[Air conditioning, Terrace]","[amex, dinersclub, mastercard, visa]",+39 0375 98351,
1981,Cavallino,corso Romita 83,Tortona,15057,Italy,€€€,Modern Cuisine,Three talented young entrepreneurs who are pas...,"[Air conditioning, Car park, Interesting wine ...","[amex, unionpay, dinersclub, discover, jcb, ma...",+39 0131 862308,http://www.cavallino-tortona.it


In [4]:
%pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## **2. Search Engine**

### **2.0. Preprocessing the Text**

In [4]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/federicolatt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and remove stopwords, then apply stemming
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

# Apply to the description field
df['processed_description'] = df['description'].apply(preprocess_text)


### **2.1. Conjunctive Query**

#### **2.1.1. Create the Index!**

In [6]:
from collections import defaultdict
import pandas as pd

vocabulary = {}
inverted_index = defaultdict(list)
term_id_counter = 0

for doc_id, description in enumerate(df['processed_description']):
    for word in description.split():
        # Map each unique word to a term_id
        if word not in vocabulary:
            vocabulary[word] = term_id_counter
            term_id_counter += 1
        term_id = vocabulary[word]
        inverted_index[term_id].append(doc_id)

# Save the vocabulary to a CSV file
pd.DataFrame(list(vocabulary.items()), columns=['term', 'term_id']).to_csv('vocabulary.csv', index=False)


In [7]:
import json

with open('inverted_index.json', 'w') as f:
    json.dump(inverted_index, f)


#### **2.1.2. Execute the Query**

In [8]:
def preprocess_query(query):
    query = query.lower()
    query = query.translate(str.maketrans('', '', string.punctuation))
    tokens = [stemmer.stem(word) for word in query.split() if word not in stop_words]
    return tokens

def conjunctive_query(query):
    query_terms = preprocess_query(query)
    term_ids = [vocabulary.get(term) for term in query_terms if term in vocabulary]

    if not term_ids:
        return pd.DataFrame(columns=["restaurantName", "address", "description", "website"])

    # Start with the document list for the first term, then intersect with others
    matching_docs = set(inverted_index[term_ids[0]])
    for term_id in term_ids[1:]:
        matching_docs &= set(inverted_index[term_id])

    results = df.loc[list(matching_docs), ["restaurantName", "address", "description", "website"]]
    return results



In [9]:
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_description'])


In [11]:
tfidf_index = defaultdict(list)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Loop over each term (feature) in the TF-IDF matrix
for term_id, term in enumerate(feature_names):
    # Get non-zero document indices and the corresponding scores for this term
    doc_indices = tfidf_matrix[:, term_id].nonzero()[0]
    scores = tfidf_matrix[:, term_id].data
    
    # Append each document ID and score to the tfidf_index dictionary 
    for doc_id, score in zip(doc_indices, scores):
        tfidf_index[term].append((doc_id, score))



### **2.2. Ranked Search Engine with TF-IDF and Cosine Similarity**

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

def ranked_query(query, top_k=5):
    query_vec = tfidf_vectorizer.transform([preprocess_text(query)])
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_doc_indices = cosine_similarities.argsort()[-top_k:][::-1]

    results = df.loc[top_doc_indices, ['restaurantName', 'address', 'description', 'website']]
    results['similarity_score'] = cosine_similarities[top_doc_indices]
    return results



### **Testing**

In [13]:
# Test the conjunctive query
query = "This pleasant, warmly decorated restaurant is ..."
conjunctive_results = conjunctive_query(query)
print(conjunctive_results)
display(conjunctive_results)
# Test the ranked query
ranked_results = ranked_query(query, top_k=5)
print(ranked_results)
display(ranked_results)


       restaurantName                         address  \
1742  Osteria Taviani  piazza Vittorio Emanuele II 28   

                                            description website  
1742  This pleasant, warmly decorated restaurant is ...          


,restaurantName,address,description,website
1742,Osteria Taviani,piazza Vittorio Emanuele II 28,"This pleasant, warmly decorated restaurant is ...",


       restaurantName                         address  \
1742  Osteria Taviani  piazza Vittorio Emanuele II 28   
867          Menzaghi             via San Giovanni 74   
768        SaleGrosso              viale II Giugno 15   
1887   Marsupino 1901               via Roma Serra 20   
1298   Lio Pellegrini               via San Tomaso 47   

                                            description  \
1742  This pleasant, warmly decorated restaurant is ...   
867   Once you’ve found the car park, make your way ...   
768   One of the town’s authentic favourites, this f...   
1887  The Marsupino family has been running this tra...   
1298  Despite its location in the town centre, the a...   

                                 website  similarity_score  
1742                                              0.302989  
867   https://www.ristorantemenzaghi.it/          0.221025  
768   http://www.ristorantesalegrosso.it          0.203840  
1887  https://www.trattoriamarsupino.it/          0.174293

,restaurantName,address,description,website,similarity_score
1742,Osteria Taviani,piazza Vittorio Emanuele II 28,"This pleasant, warmly decorated restaurant is ...",,0.302989
867,Menzaghi,via San Giovanni 74,"Once you’ve found the car park, make your way ...",https://www.ristorantemenzaghi.it/,0.221025
768,SaleGrosso,viale II Giugno 15,"One of the town’s authentic favourites, this f...",http://www.ristorantesalegrosso.it,0.203840
1887,Marsupino 1901,via Roma Serra 20,The Marsupino family has been running this tra...,https://www.trattoriamarsupino.it/,0.174293
1298,Lio Pellegrini,via San Tomaso 47,"Despite its location in the town centre, the a...",https://www.liopellegrini.it/site/,0.172042


## **3. Define a New Score!**


In [14]:
import heapq
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [15]:
def calculate_cosine_similarity(query, tfidf_vectorizer, tfidf_matrix):
    # Convert the query into a TF-IDF vector
    query_tfidf = tfidf_vectorizer.transform([query])
    
     # Compute cosine similarity between the query vector and all document vectors
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    #Return similarity scores for all documents
    return cosine_similarities

In [16]:
# Main function for custom ranking. 
def custom_scoring(query, df, tfidf_vectorizer, tfidf_matrix, top_k=5, cuisine_preferences=None, service_preferences=None, price_preferences=None):
    # Define differents weights for description, cuisine, facilities and price.
    DESCRIPTION_WEIGHT = 0.5
    CUISINE_WEIGHT = 0.2
    FACILITIES_WEIGHT = 0.2
    PRICE_WEIGHT = 0.3

    # Calculate cosine similarities between the query and all descriptions
    cosine_similarities = calculate_cosine_similarity(query, tfidf_vectorizer, tfidf_matrix)
    top_k_restaurants = []

    # Iterate over all documents and calculate the custom score for each
    for doc_id, cosine_score in enumerate(cosine_similarities):
        # Score based on description similarity
        description_score = cosine_score * DESCRIPTION_WEIGHT

        # Score for matching "cuisineType"
        cuisine_score = 0
        if 'cuisineType' in df.columns and cuisine_preferences:
            for pref in cuisine_preferences:
                if pref.lower() in df.loc[doc_id, 'cuisineType'].lower():
                    cuisine_score += CUISINE_WEIGHT

        # Score for matching "facilitiesServices"
        facilities_score = 0
        if 'facilitiesServices' in df.columns and service_preferences:
            facilities = df.loc[doc_id, 'facilitiesServices']
            if isinstance(facilities, list):
                for service in service_preferences:
                    if any(service.lower() == facility.lower() for facility in facilities):
                        facilities_score += FACILITIES_WEIGHT

        # Score for matching "priceRange"
        price_score = 0
        if 'priceRange' in df.columns and price_preferences:
            price_range = df.loc[doc_id, 'priceRange']
            for price in price_preferences:
                if price in price_range:
                    price_score += PRICE_WEIGHT

        # Calculate the final custom score
        final_score = description_score + cuisine_score + facilities_score + price_score

        # Add to the heap to maintain top-k results
        if len(top_k_restaurants) < top_k:
            heapq.heappush(top_k_restaurants, (final_score, doc_id))
        else:
            heapq.heappushpop(top_k_restaurants, (final_score, doc_id))

    # Sort the top-k results by score in descending order
    top_k_restaurants = sorted(top_k_restaurants, key=lambda x: x[0], reverse=True)

    # Prepare the final output
    results = []
    for score, doc_id in top_k_restaurants:
        results.append({
            "restaurantName": df.loc[doc_id, "restaurantName"],
            "address": df.loc[doc_id, "address"],
            "description": df.loc[doc_id, "description"],
            "website": df.loc[doc_id, "website"],
            "custom_score": round(score, 3)
        })

    return pd.DataFrame(results)

In [17]:
#Define the query and the user's preferences
query = "seafood"
cuisine_preferences = ["Modern Cuisine"]
service_preferences = ["Terrace", "Air conditioning"]   
price_preferences = ["€€€"]
top_k = 5

# Call the custom scoring function and display the results
results_df = custom_scoring(query, df, tfidf_vectorizer, tfidf_matrix, top_k, cuisine_preferences, service_preferences,  price_preferences)
display(results_df)

,restaurantName,address,description,website,custom_score
0,L'Osteria di Santa Marina,campo Santa Marina sestiere di Castello 5911,This classy inn is managed by young profession...,https://osteriadisantamarina.com/,1.011
1,Le Lampare al Fortino,via Tiepolo molo Sant'Antonio,"Built over a medieval church, this old fort th...",https://www.lelamparealfortino.it/it/home/,0.981
2,Essenza Bistrot,via delle Terme 8/A,A welcoming and original bistro-restaurant in ...,https://www.essenzabistrot.it,0.976
3,Jamantè,via San Vito 97,Located not far from the historic centre and t...,https://www.jamanteristorante.com,0.967
4,Livello 1,via Duccio di Buoninsegna 25,Situated in the suburbs off the main tourist t...,http://www.ristorantelivello1.it,0.965


## **4. Visualizing the Most Relevant Restaurants**
### **4.1. Geocode Locations**

In [18]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [19]:
# geocoder initialization
geolocator = Nominatim(user_agent="restaurantGeocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.1)

In [20]:
# DEBUG for showing the exact names of the regions

try:
    location = geocode('Volterra, Italy')
    if location:
        #print(location.address)
        print(location.raw.get('display_name', ''))
        #print((location.latitude, location.longitude))
    else:
        print("Location not found")
except Exception as e:
    print(f"Error while geocoding: {e}")

italian_regions = {"Abruzzo", "Valle d'Aosta / Vallée d'Aoste", "Puglia", "Basilicata", "Calabria", "Campania", 
                   "Emilia-Romagna", "Friuli-Venezia Giulia", "Lazio", "Liguria", "Lombardia", 
                   "Marche", "Molise", "Piemonte", "Sardigna/Sardegna", "Sicilia", "Trentino-Alto Adige/Südtirol", 
                   "Toscana", "Umbria", "Veneto"}

Volterra, Pisa, Toscana, 56048, Italia


In [21]:
regions = {}
coordinates = {}

# Extraction of the region
def extract_region(display_name):
    parts = set(display_name.split(', '))
    regions = list(parts.intersection(italian_regions))[0]
    return regions

for city in df['city'].unique():
    try:
        location = geocode(city + ', Italy')
        if location:
            print(f"YES! Location found for city: {city}")
            display_name = location.raw.get('display_name', '')
            regions[city] = extract_region(display_name)
            coordinates[city] = (location.latitude, location.longitude)
        else:
            print(f"Location not found for city: {city}")
            # regions[city] = 'Unknown'
            # coordinates[city] = (None, None)
    except Exception as e:
        print(f"Error while geocoding {city}: {e}")
        # regions[city] = 'Unknown'
        # coordinates[city] = (None, None)

# Adding regions and coordinates to the dataframe
df['region'] = df['city'].map(regions)
df['coordinates'] = df['city'].map(coordinates)

df.insert(df.columns.get_loc('city') + 1, 'region', df.pop('region'))
df.insert(df.columns.get_loc('city') + 2, 'coordinates', df.pop('coordinates'))

YES! Location found for city: Capolago
YES! Location found for city: Milan
YES! Location found for city: Mules
YES! Location found for city: Forlì
YES! Location found for city: Naples
YES! Location found for city: Zelarino
YES! Location found for city: Rome
YES! Location found for city: Marzocca
YES! Location found for city: Positano
YES! Location found for city: Venice
YES! Location found for city: Tavarnelle Val di Pesa
YES! Location found for city: Nals
YES! Location found for city: Turin
YES! Location found for city: Sernaglia della Battaglia
YES! Location found for city: Spoleto
YES! Location found for city: Palazzolo sull'Oglio
YES! Location found for city: San Donato in Poggio
YES! Location found for city: Verona
YES! Location found for city: Giustino
YES! Location found for city: Labico
YES! Location found for city: Fano
YES! Location found for city: Villa d'Almè
YES! Location found for city: Mazzorbo
YES! Location found for city: Vallesaccarda
YES! Location found for city: Bes

In [22]:
# DEBUG

# for k,v in regions.items():
#     print(f"({k}, {v})")
#     print(f"({k}, {coordinates[k]})")
#     print()

In [35]:
regions_list = [
    (city, regions[city], coordinates[city][0], coordinates[city][1])
    for city in regions.keys() & coordinates.keys()
]

In [24]:
# for element in regions_list:
#     print(element)

In [36]:
display(df)

,restaurantName,address,city,region,coordinates,postalCode,country,priceRange,priceValue,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,processed_description
0,Da Annetta,via Fè 25,Capolago,Lombardia,"(45.7936297, 8.8096689)",21100,Italy,€€,2,"Traditional Cuisine, Modern Cuisine","In an 18th Century building, a rustic yet eleg...","[Air conditioning, Car park, Interesting wine ...","[amex, mastercard, visa]",+39 0332 490230,https://www.ristoranteannetta.it/,18th centuri build rustic yet eleg atmospher p...
1,Osaka,corso Giuseppe Garibaldi 68,Milan,Lombardia,"(45.4641943, 9.1896346)",20121,Italy,€€€,3,"Japanese, Traditional Cuisine",Tucked away in a short arcade on the old road ...,"[Air conditioning, Counter dining, Notable sak...","[amex, unionpay, maestrocard, mastercard, visa]",+39 02 2906 0678,https://www.milanoosaka.com,tuck away short arcad old road milan como rest...
2,Gourmetstube Einhorn,Campo di Trens,Mules,Trentino-Alto Adige/Südtirol,"(46.8549387, 11.5238006)",39040,Italy,€€€€,4,"Creative, International","Occupying a 13C post house, this restaurant is...","[Car park, Garden or park, Interesting wine list]","[dinersclub, maestrocard, mastercard, visa]",+39 0472 771136,https://www.stafler.com/it/hotel-gourmet-alto-...,occupi 13c post hous restaur part eleg stafler...
3,Benso,piazza Cavour 7,Forlì,Emilia-Romagna,"(44.2227278, 12.041273)",47121,Italy,€€,2,"Modern Cuisine, Italian Contemporary",Situated in a small garden in the historic cen...,"[Air conditioning, Terrace, Wheelchair access]","[amex, mastercard, visa]",+39 346 116 7238,https://www.bensofood.com/,situat small garden histor centr modern bistro...
4,La Notizia 53,via Caravaggio 53/55,Naples,Campania,"(40.8358846, 14.2487679)",80126,Italy,€,1,Pizza,Although situated away from the tourist centre...,[Air conditioning],"[amex, maestrocard, mastercard, visa]",+39 081 714 2155,http://www.pizzarialanotizia.com,although situat away tourist centr enzo coccia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,Foresta,via Litoranea 2,Marina di Pisa,Toscana,"(43.6724218, 10.2727668)",56128,Italy,€€€,3,"Seafood, Classic Cuisine","Overlooking the Tyrrhenian Sea, all the tables...","[Air conditioning, Great view, Terrace, Wheelc...","[amex, mastercard, visa]",+39 050 35082,https://www.ristoranteforesta.com/,overlook tyrrhenian sea tabl restaur offer sea...
1979,Oberraindlhof,Raindl 49,Madonna di Senales,Trentino-Alto Adige/Südtirol,"(46.722537, 10.858376644500439)",39020,Italy,€€,2,"Traditional Cuisine, Regional Cuisine",The family that run this restaurant occupying ...,"[Car park, Great view, Interesting wine list, ...","[amex, maestrocard, mastercard, visa]",+39 0473 679131,https://www.oberraindlhof.com/it/hotel-per-buo...,famili run restaur occupi old farmhous origin ...
1980,Antica Trattoria Gianna,via Maggiore 12,Recorfano,Lombardia,"(45.1036904, 10.3451948)",26030,Italy,€,1,"Lombardian, Country cooking","A delightful family trattoria offering simple,...","[Air conditioning, Terrace]","[amex, dinersclub, mastercard, visa]",+39 0375 98351,,delight famili trattoria offer simpl reason pr...
1981,Cavallino,corso Romita 83,Tortona,Piemonte,"(44.89666, 8.8641282)",15057,Italy,€€€,3,Modern Cuisine,Three talented young entrepreneurs who are pas...,"[Air conditioning, Car park, Interesting wine ...","[amex, unionpay, dinersclub, discover, jcb, ma...",+39 0131 862308,http://www.cavallino-tortona.it,three talent young entrepreneur passion good f...


### **4.2. Ask a Large Language Model (LLM)**

In [26]:
# for this purpose, We've done point 4.1.

### **4.3. Map Setup**

In [27]:
import folium

In [48]:
# Creates a map centered on Italy
italy_center = [41.8719, 12.5674]  # Latitude and longitude of Italy
restaurantMap = folium.Map(location=italy_center, zoom_start=6)

# Adds markers to the map
for city, region, lat, lon in regions_list:
    folium.Marker(
        location=[lat, lon],
        popup=f"{city}, {region}",
        tooltip=f"{city}",
    ).add_to(restaurantMap)

# saves and show the map
restaurantMap.save("italy_city_restaurant_map.html")
restaurantMap

### **4.4. Encoding Price Ranges**

In [49]:
# mapping of price ranges and corresponding numeric values
price_values = {'€': 1, '€€': 2, '€€€': 3, '€€€€': 4}

# mapping of numeric values and corresponding colors
price_colors = {1: 'green', 2: 'orange', 3: 'red', 4: 'darkred'}

# conversion of price ranges to numeric values into the dataframe
df['priceValue'] = df['priceRange'].map(price_values)
df.insert(df.columns.get_loc('priceRange') + 1, 'priceValue', df.pop('priceValue'))

In [50]:
display(df)

,restaurantName,address,city,region,coordinates,postalCode,country,priceRange,priceValue,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,processed_description
0,Da Annetta,via Fè 25,Capolago,Lombardia,"(45.7936297, 8.8096689)",21100,Italy,€€,2,"Traditional Cuisine, Modern Cuisine","In an 18th Century building, a rustic yet eleg...","[Air conditioning, Car park, Interesting wine ...","[amex, mastercard, visa]",+39 0332 490230,https://www.ristoranteannetta.it/,18th centuri build rustic yet eleg atmospher p...
1,Osaka,corso Giuseppe Garibaldi 68,Milan,Lombardia,"(45.4641943, 9.1896346)",20121,Italy,€€€,3,"Japanese, Traditional Cuisine",Tucked away in a short arcade on the old road ...,"[Air conditioning, Counter dining, Notable sak...","[amex, unionpay, maestrocard, mastercard, visa]",+39 02 2906 0678,https://www.milanoosaka.com,tuck away short arcad old road milan como rest...
2,Gourmetstube Einhorn,Campo di Trens,Mules,Trentino-Alto Adige/Südtirol,"(46.8549387, 11.5238006)",39040,Italy,€€€€,4,"Creative, International","Occupying a 13C post house, this restaurant is...","[Car park, Garden or park, Interesting wine list]","[dinersclub, maestrocard, mastercard, visa]",+39 0472 771136,https://www.stafler.com/it/hotel-gourmet-alto-...,occupi 13c post hous restaur part eleg stafler...
3,Benso,piazza Cavour 7,Forlì,Emilia-Romagna,"(44.2227278, 12.041273)",47121,Italy,€€,2,"Modern Cuisine, Italian Contemporary",Situated in a small garden in the historic cen...,"[Air conditioning, Terrace, Wheelchair access]","[amex, mastercard, visa]",+39 346 116 7238,https://www.bensofood.com/,situat small garden histor centr modern bistro...
4,La Notizia 53,via Caravaggio 53/55,Naples,Campania,"(40.8358846, 14.2487679)",80126,Italy,€,1,Pizza,Although situated away from the tourist centre...,[Air conditioning],"[amex, maestrocard, mastercard, visa]",+39 081 714 2155,http://www.pizzarialanotizia.com,although situat away tourist centr enzo coccia...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,Foresta,via Litoranea 2,Marina di Pisa,Toscana,"(43.6724218, 10.2727668)",56128,Italy,€€€,3,"Seafood, Classic Cuisine","Overlooking the Tyrrhenian Sea, all the tables...","[Air conditioning, Great view, Terrace, Wheelc...","[amex, mastercard, visa]",+39 050 35082,https://www.ristoranteforesta.com/,overlook tyrrhenian sea tabl restaur offer sea...
1979,Oberraindlhof,Raindl 49,Madonna di Senales,Trentino-Alto Adige/Südtirol,"(46.722537, 10.858376644500439)",39020,Italy,€€,2,"Traditional Cuisine, Regional Cuisine",The family that run this restaurant occupying ...,"[Car park, Great view, Interesting wine list, ...","[amex, maestrocard, mastercard, visa]",+39 0473 679131,https://www.oberraindlhof.com/it/hotel-per-buo...,famili run restaur occupi old farmhous origin ...
1980,Antica Trattoria Gianna,via Maggiore 12,Recorfano,Lombardia,"(45.1036904, 10.3451948)",26030,Italy,€,1,"Lombardian, Country cooking","A delightful family trattoria offering simple,...","[Air conditioning, Terrace]","[amex, dinersclub, mastercard, visa]",+39 0375 98351,,delight famili trattoria offer simpl reason pr...
1981,Cavallino,corso Romita 83,Tortona,Piemonte,"(44.89666, 8.8641282)",15057,Italy,€€€,3,Modern Cuisine,Three talented young entrepreneurs who are pas...,"[Air conditioning, Car park, Interesting wine ...","[amex, unionpay, dinersclub, discover, jcb, ma...",+39 0131 862308,http://www.cavallino-tortona.it,three talent young entrepreneur passion good f...


In [108]:
# creates a new df for the coordinates (city, latitude, longitude)
df_coordinates = df[['city', 'coordinates']].copy()
df_coordinates[['latitude', 'longitude']] = df_coordinates['coordinates'].apply(pd.Series)
df_coordinates.drop(columns='coordinates', inplace=True)

# calculate the mean of price ranges for each city
df_city_price_mean = df.groupby('city')['priceValue'].mean().reset_index().rename(columns={'priceValue': 'priceRangeMean'})
df_city_price_mean['color'] = df_city_price_mean['priceRangeMean'].round().map(price_colors)

#display(df_city_price_mean)

# we union data on 'city' column
df_merged = pd.merge(df_city_price_mean, df_coordinates, on='city', how='left')
df_merged = df_merged.drop_duplicates(subset='city').reset_index(drop=True)

# we want to add in the final df all the informations we need inside the new map, se we add also regions
df_regions = df[['city', 'region']].copy()

# merge of the regions and positioning the field right after th cities one
df_merged = pd.merge(df_merged, df_regions, on='city', how='left')
df_merged.insert(df_merged.columns.get_loc('city') + 1, 'region', df_merged.pop('region'))

# we filter the df to remove the NAN values of latitude and longitude
df_filtered = df_merged.dropna(subset=['latitude', 'longitude'])
# we filter further the df to also remove the duplicates cities
df_filtered = df_filtered.drop_duplicates(subset='city').reset_index()

display(df_filtered)

,index,city,region,priceRangeMean,color,latitude,longitude
0,0,Abano Terme,Veneto,2.000000,orange,45.360314,11.789783
1,1,Aci Castello,Sicilia,2.000000,orange,37.554211,15.148560
2,2,Acqualagna,Marche,2.000000,orange,43.625884,12.675974
3,3,Acri,Calabria,1.000000,green,39.493005,16.384950
4,4,Acuto,Lazio,4.000000,darkred,41.791521,13.174394
...,...,...,...,...,...,...,...
1103,1975,Vodo di Cadore,Veneto,2.333333,orange,46.419622,12.244899
1104,1978,Voghera,Lombardia,2.000000,orange,44.992670,9.008667
1105,1979,Volterra,Toscana,2.500000,orange,43.400337,10.860019
1106,1981,Zafferana Etnea,Sicilia,3.000000,red,37.693285,15.106460


In [109]:
# creates another map centered on Italy
coloredRestaurantMap = folium.Map(location=italy_center, zoom_start=6)

# adds markers to the map
for _, row in df_filtered.iterrows():
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = f"{row['city']}, {row['region']}", #: {row['priceRangeMean']:.2f}",
        tooltip=f"{row['city']}",
        icon=folium.Icon(color=row['color'])
    ).add_to(coloredRestaurantMap)

# adding a legend to the map
legend_html = '''
<div style="position: fixed; 
            top: 50px; right: 50px; width: 130px; height: 160px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 10px;">
    <b>Price ranges</b><br>
    <i class="fa fa-map-marker fa-2x" style="color:#70ad25; vertical-align: middle; margin-top: 2px; margin-bottom: 1px"></i> €<br>
    <i class="fa fa-map-marker fa-2x" style="color:#f59630; vertical-align: middle; margin-bottom: 1px;"></i> €€<br>
    <i class="fa fa-map-marker fa-2x" style="color:#d53e2a; vertical-align: middle; margin-bottom: 1px;"></i> €€€<br>
    <i class="fa fa-map-marker fa-2x" style="color:#9e3235; vertical-align: middle; margin-bottom: 1px;"></i> €€€€<br>
</div>
'''
coloredRestaurantMap.get_root().html.add_child(folium.Element(legend_html))

# save and show the map
coloredRestaurantMap.save('colored_italy_city_restaurant_map.html')
coloredRestaurantMap

### **4.5. Plot Top-K Restaurants**

In [110]:
# adding for convenience, to the original dataframe, latitude and longitude separately
df[['latitude', 'longitude']] = df['coordinates'].apply(pd.Series)
df.insert(df.columns.get_loc('coordinates') + 1, 'latitude', df.pop('latitude'))
df.insert(df.columns.get_loc('coordinates') + 2, 'longitude', df.pop('longitude'))
#df

In [111]:
results_df # the resulting df from point 3...

,restaurantName,address,description,website,custom_score
0,L'Osteria di Santa Marina,campo Santa Marina sestiere di Castello 5911,This classy inn is managed by young profession...,https://osteriadisantamarina.com/,1.011
1,Le Lampare al Fortino,via Tiepolo molo Sant'Antonio,"Built over a medieval church, this old fort th...",https://www.lelamparealfortino.it/it/home/,0.981
2,Essenza Bistrot,via delle Terme 8/A,A welcoming and original bistro-restaurant in ...,https://www.essenzabistrot.it,0.976
3,Jamantè,via San Vito 97,Located not far from the historic centre and t...,https://www.jamanteristorante.com,0.967
4,Livello 1,via Duccio di Buoninsegna 25,Situated in the suburbs off the main tourist t...,http://www.ristorantelivello1.it,0.965


In [112]:
# ...and for plotting, we need for each restaurant, his city, region, latitude and longitude
df_top_k = pd.merge(results_df, df, on='restaurantName', how='left')

# keeps only one of duplicates columns and renames it
df_top_k = df_top_k.drop(columns=['address_y', 'description_y', 'website_y'])
df_top_k = df_top_k.rename(columns={'address_x': 'address', 'description_x': 'description', 'website_x': 'website'})

# keeps only the columns of interest
df_top_k = df_top_k[['restaurantName', 'address', 'description', 'website', 'custom_score', 'city', 'region', 'latitude', 'longitude']]

df_top_k 

,restaurantName,address,description,website,custom_score,city,region,latitude,longitude
0,L'Osteria di Santa Marina,campo Santa Marina sestiere di Castello 5911,This classy inn is managed by young profession...,https://osteriadisantamarina.com/,1.011,Venice,Veneto,45.437191,12.334590
1,Le Lampare al Fortino,via Tiepolo molo Sant'Antonio,"Built over a medieval church, this old fort th...",https://www.lelamparealfortino.it/it/home/,0.981,Trani,Puglia,41.275088,16.416259
2,Essenza Bistrot,via delle Terme 8/A,A welcoming and original bistro-restaurant in ...,https://www.essenzabistrot.it,0.976,Olbia,Sardigna/Sardegna,40.923266,9.502744
3,Jamantè,via San Vito 97,Located not far from the historic centre and t...,https://www.jamanteristorante.com,0.967,Polignano a Mare,Puglia,40.994446,17.222488
4,Livello 1,via Duccio di Buoninsegna 25,Situated in the suburbs off the main tourist t...,http://www.ristorantelivello1.it,0.965,Rome,Lazio,41.893320,12.482932


In [113]:
top_k_map = folium.Map(location=italy_center, zoom_start=6)

# add markers for the top-k restaurants
for _, row in df_top_k.iterrows():
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = f"""{row['restaurantName']}<br> Score: {row['custom_score']}<br><br> {row['address']}<br> 
                    {row['city']}, {row['region']}<br><br> {row['description']}<br><br> {row['website']}""",
        tooltip=f"{row['restaurantName']}<br> Score: {row['custom_score']}",
        icon=folium.Icon(color='cadetblue')
    ).add_to(top_k_map)

# save and display the map
top_k_map.save('top_k_restaurant_map.html')
top_k_map

## **5. Bonus**


In [33]:
from collections import defaultdict
import pandas as pd

# Funzione per preprocessare il testo (es. tokenizzazione e normalizzazione)
def preprocess_text(text):
    if pd.isnull(text):
        return []
    # Tokenizzazione semplice e conversione in minuscolo
    return [token.strip().lower() for token in text.split()]

# Funzione per creare vocabolari e indici invertiti
def create_vocabularies_and_indexes(data, fields):
    vocabularies = {field: {} for field in fields}
    inverted_indexes = {field: defaultdict(set) for field in fields}
    term_ids = {field: 0 for field in fields}
    
    for doc_id, row in data.iterrows():
        for field in fields:
            tokens = preprocess_text(row[field])
            for token in tokens:
                if token not in vocabularies[field]:
                    vocabularies[field][token] = term_ids[field]
                    term_ids[field] += 1
                term_id = vocabularies[field][token]
                inverted_indexes[field][term_id].add(doc_id)
    
    return vocabularies, inverted_indexes

# Funzione per eseguire una query congiunta
def conjunctive_query(query, vocabulary, inverted_index):
    query_tokens = preprocess_text(query)
    query_term_ids = [vocabulary[token] for token in query_tokens if token in vocabulary]
    
    if not query_term_ids:
        return set()
    
    doc_lists = [inverted_index[tid] for tid in query_term_ids if tid in inverted_index]
    return set.intersection(*doc_lists) if doc_lists else set()

# Funzione avanzata per ricerca multi-campo
def advanced_search_multi_field(data, vocabularies, inverted_indexes, queries):
    results = set(data.index)
    
    for field, query in queries.items():
        if query:
            field_vocab = vocabularies[field]
            field_index = inverted_indexes[field]
            field_results = conjunctive_query(query, field_vocab, field_index)
            results.intersection_update(field_results)
    
    return data.loc[list(results), ['restaurantName', 'city', 'cuisineType', 'priceRange', 
                                    'creditCards', 'facilitiesServices', 'country', 'website']]

# Funzione per applicare filtri
def apply_filters(data, price_range=None, region=None, facilities=None, credit_cards=None):
    if price_range:
        data = data[data['priceRange'].isin(price_range)]
    if region:
        data = data[data['country'].isin(region)]
    if facilities:
        data = data[data['facilitiesServices'].apply(lambda x: any(f in x for f in facilities))]
    if credit_cards:
        data = data[data['creditCards'].apply(lambda x: any(cc in x for cc in credit_cards))]
    return data

# Creazione degli indici dai dati estratti
fields_to_index = ['restaurantName', 'city', 'cuisineType']
vocabularies, inverted_indexes = create_vocabularies_and_indexes(df, fields_to_index)

# Input dell'utente
queries = {
    'restaurantName': input("Inserisci il nome del ristorante: ").strip(),
    'city': input("Inserisci la città: ").strip(),
    'cuisineType': input("Inserisci il tipo di cucina: ").strip()
}

# Filtri aggiuntivi
price_range = input("Inserisci la fascia di prezzo (es. €, €€, separati da virgola): ").strip().split(",")
region = input("Inserisci il paese o regione (es. Italia, Francia): ").strip().split(",")
facilities = input("Inserisci i servizi (es. WiFi, Accessibile): ").strip().split(",")
credit_cards = input("Inserisci le carte di credito accettate (es. Visa, MasterCard): ").strip().split(",")

# Ricerca e filtri
search_results = advanced_search_multi_field(df, vocabularies, inverted_indexes, queries)
filtered_results = apply_filters(search_results, price_range, region, facilities, credit_cards)

# Output dei risultati
if filtered_results.empty:
    print("Nessun risultato trovato.")
else:
    print("Risultati trovati:")
    print(filtered_results.to_string(index=False))


KeyError: 'creditCards'

In [ ]:
from collections import defaultdict
import pandas as pd

def preprocess_text(text):
    """
    Preprocesses the input text by tokenizing and normalizing it.
    Example implementation: Convert text to lowercase and split on spaces.
    """
    return text.lower().split()

def build_vocabulary_and_inverted_indexes(df):
    """
    Builds vocabularies and inverted indexes for the fields: restaurantName, city, and cuisineType.
    """
    vocab = {
        'restaurantName': {},
        'city': {},
        'cuisineType': {}
    }
    
    inverted_index = {
        'restaurantName': defaultdict(set),
        'city': defaultdict(set),
        'cuisineType': defaultdict(set)
    }
    
    term_counter = {
        'restaurantName': 0,
        'city': 0,
        'cuisineType': 0
    }

    # Iterate through each row in the dataset
    for record_id, row in df.iterrows():
        for field in ['restaurantName', 'city', 'cuisineType']:
            tokens = preprocess_text(row[field])  # Tokenize the field value

            for token in tokens:
                # If the token doesn't exist in the vocabulary, add it and assign a new term ID
                if token not in vocab[field]:
                    vocab[field][token] = term_counter[field]
                    term_counter[field] += 1

                # Get the term ID for the token and add the current record to the inverted index
                term_id = vocab[field][token]
                inverted_index[field][term_id].add(record_id)

    return vocab, inverted_index

def conjunctive_query(query, vocab, inverted_index):
    """
    Performs a conjunctive query to retrieve documents containing all terms from the query.
    """
    # Preprocess the query text and convert it to term IDs using the vocabulary
    query_tokens = preprocess_text(query)
    query_term_ids = [vocab.get(token) for token in query_tokens if token in vocab]

    # If no terms from the query are found in the vocabulary, return an empty set
    if not query_term_ids:
        return set()

    # Find document IDs containing all term IDs from the query
    doc_sets = [inverted_index[term_id] for term_id in query_term_ids if term_id in inverted_index]

    # If doc_sets are not empty, find the intersection of all document sets
    if doc_sets:
        matching_docs = set.intersection(*doc_sets)
    else:
        matching_docs = set()

    # Return the set of matching document indices
    return matching_docs

def advanced_search_with_filters(data, name_filter='', city_filter='', cuisine_filter=''):
    """
    Performs an advanced search with filters on restaurant name, city, and cuisine type.
    """
    # Generate vocabularies and inverted indexes for each field
    vocab, inverted_idx = build_vocabulary_and_inverted_indexes(data)
    
    # Start with a set containing all document indices
    matching_docs = set(data.index)

    # Search by `restaurantName` if a `name_filter` is provided
    if name_filter:
        name_vocab, name_inv_idx = vocab['restaurantName'], inverted_idx['restaurantName']
        name_results = conjunctive_query(name_filter, name_vocab, name_inv_idx)
        matching_docs.intersection_update(name_results)

    # Search by `city` if a `city_filter` is provided
    if city_filter:
        city_vocab, city_inv_idx = vocab['city'], inverted_idx['city']
        city_results = conjunctive_query(city_filter, city_vocab, city_inv_idx)
        matching_docs.intersection_update(city_results)

    # Search by `cuisineType` if a `cuisine_filter` is provided
    if cuisine_filter:
        cuisine_vocab, cuisine_inv_idx = vocab['cuisineType'], inverted_idx['cuisineType']
        cuisine_results = conjunctive_query(cuisine_filter, cuisine_vocab, cuisine_inv_idx)
        matching_docs.intersection_update(cuisine_results)

    # Return the filtered data with the relevant columns
    filtered_restaurants = data.loc[list(matching_docs), ['restaurantName', 'address', 'cuisineType', 'priceRange', 'website']]
    return filtered_restaurants


# Prompt user for search filters
name_filter = input("Enter restaurant name to search: ")
city_filter = input("Enter city to search: ")
cuisine_filter = input("Enter cuisine type to search: ")

# Execute the advanced search with filters
filtered_results = advanced_search_with_filters(data, name_filter, city_filter, cuisine_filter)

# Display the filtered results
print(filtered_results)


Empty DataFrame
Columns: [restaurantName, address, cuisineType, priceRange, website]
Index: []


In [ ]:
from collections import defaultdict
import pandas as pd
import os
from bs4 import BeautifulSoup


def preprocess_text_case_insensitive(text):
    """
    Preprocesses the input text by converting it to lowercase and splitting it into tokens.
    """
    return text.lower().split()


def build_vocabulary_and_inverted_indexes_case_insensitive(df):
    """
    Builds vocabularies and inverted indexes for fields in the dataset, considering case insensitivity.
    """
    vocab = {
        'restaurantName': {},
        'city': {},
        'cuisineType': {}
    }
    
    inverted_index = {
        'restaurantName': defaultdict(set),
        'city': defaultdict(set),
        'cuisineType': defaultdict(set)
    }
    
    term_counter = {
        'restaurantName': 0,
        'city': 0,
        'cuisineType': 0
    }

    for record_id, row in df.iterrows():
        for field in ['restaurantName', 'city', 'cuisineType']:
            tokens = preprocess_text_case_insensitive(row[field]) if pd.notnull(row[field]) else []

            for token in tokens:
                if token not in vocab[field]:
                    vocab[field][token] = term_counter[field]
                    term_counter[field] += 1

                term_id = vocab[field][token]
                inverted_index[field][term_id].add(record_id)

    return vocab, inverted_index


def conjunctive_query_case_insensitive(query, vocab, inverted_index):
    """
    Performs a conjunctive query that is case-insensitive.
    """
    query_tokens = preprocess_text_case_insensitive(query)
    query_term_ids = [vocab.get(token) for token in query_tokens if token in vocab]

    if not query_term_ids:
        return set()

    doc_sets = [inverted_index[term_id] for term_id in query_term_ids if term_id in inverted_index]

    return set.intersection(*doc_sets) if doc_sets else set()


def advanced_search_with_filters_case_insensitive(data, name_filter='', city_filter='', cuisine_filter=''):
    """
    Performs an advanced search with filters on restaurant name, city, and cuisine type.
    Search is case-insensitive.
    """
    vocab, inverted_idx = build_vocabulary_and_inverted_indexes_case_insensitive(data)
    matching_docs = set(data.index)

    if name_filter:
        name_vocab, name_inv_idx = vocab['restaurantName'], inverted_idx['restaurantName']
        name_results = conjunctive_query_case_insensitive(name_filter, name_vocab, name_inv_idx)
        matching_docs.intersection_update(name_results)

    if city_filter:
        city_vocab, city_inv_idx = vocab['city'], inverted_idx['city']
        city_results = conjunctive_query_case_insensitive(city_filter, city_vocab, city_inv_idx)
        matching_docs.intersection_update(city_results)

    if cuisine_filter:
        cuisine_vocab, cuisine_inv_idx = vocab['cuisineType'], inverted_idx['cuisineType']
        cuisine_results = conjunctive_query_case_insensitive(cuisine_filter, cuisine_vocab, cuisine_inv_idx)
        matching_docs.intersection_update(cuisine_results)

    filtered_restaurants = data.loc[list(matching_docs), ['restaurantName', 'address', 'city', 'priceRange', 'cuisineType', 'website']]
    return filtered_restaurants


# Esegui il processo per estrarre i dati dai file HTML
def extract_restaurant_details(content):
    """
    Extracts details about a restaurant from the HTML content.
    """
    name = content.find('h1', class_='data-sheet__title').get_text(strip=True) if content.find('h1', class_='data-sheet__title') else ""
    firstRow = content.find_all("div", class_="data-sheet__block--text")[0].get_text(strip=True)
    firstRow_list = [info.strip() for info in firstRow.split(",")]

    address = " ".join(firstRow_list[:-3]) if len(firstRow_list) > 3 else ""
    city = firstRow_list[-3] if len(firstRow_list) > 2 else ""
    postalCode = firstRow_list[-2] if len(firstRow_list) > 1 else ""
    country = firstRow_list[-1] if firstRow_list else ""

    secondRow = content.find_all("div", class_="data-sheet__block--text")[1].get_text(strip=True)
    secondRow_list = [info.strip() for info in secondRow.split("·")]

    priceRange = secondRow_list[0] if secondRow_list else ""
    cuisineType = secondRow_list[1] if len(secondRow_list) > 1 else ""

    description = content.find("div", class_="data-sheet__description").get_text(strip=True) if content.find("div", class_="data-sheet__description") else ""
    facilitiesServices_div = content.find_all("div", class_="col col-12 col-lg-6")
    facilitiesServices = [li.get_text(strip=True) for li in facilitiesServices_div[0].find_all("li")] if facilitiesServices_div else []
    creditCards_div = content.find("div", class_="restaurant-details__services--info")
    creditCards = [os.path.basename(img["data-src"]).split("-")[0] for img in creditCards_div.find_all("img")] if creditCards_div else []
    phoneNumber = content.find("span", attrs={"x-ms-format-detection": "none"}).get_text(strip=True) if content.find("span", attrs={"x-ms-format-detection": "none"}) else ""
    website_div = content.find("div", class_="collapse__block-item link-item")
    website = website_div.find("a", class_="link js-dtm-link")["href"] if website_div and website_div.find("a", class_="link js-dtm-link") else ""

    return {
        "restaurantName": name,
        "address": address,
        "city": city,
        "postalCode": postalCode,
        "country": country,
        "priceRange": priceRange,
        "cuisineType": cuisineType,
        "description": description,
        "facilitiesServices": facilitiesServices,
        "creditCards": creditCards,
        "phoneNumber": phoneNumber,
        "website": website
    }


# Carica i dati
dir_paths = [os.path.join('pages', dir) for dir in os.listdir('pages')]
data = []
for dir in dir_paths:
    for html_file in os.listdir(dir):
        if html_file.endswith(".html"):
            with open(os.path.join(dir, html_file), "r", encoding="utf-8") as file:
                soup = BeautifulSoup(file, "html.parser")
                restaurant_details = extract_restaurant_details(soup)
                data.append(restaurant_details)

df = pd.DataFrame(data)

# Filtri di esempio
name_filter = input("Enter restaurant name: ")
city_filter = input("Enter city: ")
cuisine_filter = input("Enter cuisine type: ")

filtered_results = advanced_search_with_filters_case_insensitive(df, name_filter, city_filter, cuisine_filter)
print(filtered_results)


    restaurantName                            address       city priceRange  \
358           Dama  Via Mulino località San Gaudenzio  Cervesina        €€€   

        cuisineType                          website  
358  Modern Cuisine  https://www.hcsg.it/ristorante/  


In [ ]:
display(filtered_results)

,restaurantName,address,city,priceRange,cuisineType,website
358,Dama,Via Mulino località San Gaudenzio,Cervesina,€€€,Modern Cuisine,https://www.hcsg.it/ristorante/


In [ ]:
def advanced_search_with_filters_enhanced(data, name_filter='', city_filter='', cuisine_filter='', 
                                          price_range_filter=None, region_filter=None, 
                                          credit_card_filter=None, facilities_filter=None):
    """
    Performs an advanced search with additional filters for price range, region, credit cards, and facilities.
    """
    vocab, inverted_idx = build_vocabulary_and_inverted_indexes_case_insensitive(data)
    matching_docs = set(data.index)

    # Filtri base
    if name_filter:
        name_vocab, name_inv_idx = vocab['restaurantName'], inverted_idx['restaurantName']
        name_results = conjunctive_query_case_insensitive(name_filter, name_vocab, name_inv_idx)
        matching_docs.intersection_update(name_results)

    if city_filter:
        city_vocab, city_inv_idx = vocab['city'], inverted_idx['city']
        city_results = conjunctive_query_case_insensitive(city_filter, city_vocab, city_inv_idx)
        matching_docs.intersection_update(city_results)

    if cuisine_filter:
        cuisine_vocab, cuisine_inv_idx = vocab['cuisineType'], inverted_idx['cuisineType']
        cuisine_results = conjunctive_query_case_insensitive(cuisine_filter, cuisine_vocab, cuisine_inv_idx)
        matching_docs.intersection_update(cuisine_results)

    # Filtro per fascia di prezzo
    if price_range_filter:
        price_docs = set(data[data['priceRange'].isin(price_range_filter)].index)
        matching_docs.intersection_update(price_docs)

    # Filtro per regione
    if region_filter:
        region_docs = set(data[data['country'].str.contains('|'.join(region_filter), case=False, na=False)].index)
        matching_docs.intersection_update(region_docs)

    # Filtro per carte di credito accettate
    if credit_card_filter:
        credit_docs = set(data[data['creditCards'].apply(lambda cards: any(card in cards for card in credit_card_filter))].index)
        matching_docs.intersection_update(credit_docs)

    # Filtro per servizi e strutture
    if facilities_filter:
        facilities_docs = set(data[data['facilitiesServices'].apply(lambda facilities: all(facility in facilities for facility in facilities_filter))].index)
        matching_docs.intersection_update(facilities_docs)

    # Restituzione dei ristoranti filtrati
    filtered_restaurants = data.loc[list(matching_docs), ['restaurantName', 'address', 'city', 'priceRange', 'cuisineType', 'website']]
    return filtered_restaurants


# Interazione con l'utente per i nuovi filtri
name_filter = input("Enter restaurant name: ")
city_filter = input("Enter city: ")
cuisine_filter = input("Enter cuisine type: ")

# Filtro per fascia di prezzo
price_range_filter = input("Enter price range (comma-separated, e.g., €€ or €€€): ").split(',') if input("Filter by price range? (yes/no): ").lower() == 'yes' else None

# Filtro per regione
region_filter = input("Enter Italian regions (comma-separated): ").split(',') if input("Filter by region? (yes/no): ").lower() == 'yes' else None

# Filtro per carte di credito
credit_card_filter = input("Enter accepted credit card types (comma-separated, e.g., Visa, MasterCard): ").split(',') if input("Filter by credit cards? (yes/no): ").lower() == 'yes' else None

# Filtro per servizi
facilities_filter = input("Enter required facilities/services (comma-separated, e.g., Wi-Fi, Parking): ").split(',') if input("Filter by facilities? (yes/no): ").lower() == 'yes' else None

# Esegui la ricerca avanzata con tutti i filtri
filtered_results = advanced_search_with_filters_enhanced(
    df,
    name_filter=name_filter,
    city_filter=city_filter,
    cuisine_filter=cuisine_filter,
    price_range_filter=price_range_filter,
    region_filter=region_filter,
    credit_card_filter=credit_card_filter,
    facilities_filter=facilities_filter
)

# Visualizzazione dei risultati
print(filtered_results)


     restaurantName         address      city priceRange  \
356  O Me O Il Mare  Via Roma 45/47  Gragnano       €€€€   

                              cuisineType                website  
356  Italian Contemporary, Modern Cuisine  http://omeoilmare.com  


In [ ]:
def advanced_search_with_filters_enhanced(data, name_filter='', city_filter='', cuisine_filter='', 
                                          price_range_filter=None, region_filter=None, 
                                          credit_card_filter=None, facilities_filter=None):
    """
    Performs an advanced search with additional filters for price range, region, credit cards, and facilities.
    """
    vocab, inverted_idx = build_vocabulary_and_inverted_indexes_case_insensitive(data)
    matching_docs = set(data.index)

    # Filtri base
    if name_filter:
        name_vocab, name_inv_idx = vocab['restaurantName'], inverted_idx['restaurantName']
        name_results = conjunctive_query_case_insensitive(name_filter, name_vocab, name_inv_idx)
        matching_docs.intersection_update(name_results)

    if city_filter:
        city_vocab, city_inv_idx = vocab['city'], inverted_idx['city']
        city_results = conjunctive_query_case_insensitive(city_filter, city_vocab, city_inv_idx)
        matching_docs.intersection_update(city_results)

    if cuisine_filter:
        cuisine_vocab, cuisine_inv_idx = vocab['cuisineType'], inverted_idx['cuisineType']
        cuisine_results = conjunctive_query_case_insensitive(cuisine_filter, cuisine_vocab, cuisine_inv_idx)
        matching_docs.intersection_update(cuisine_results)

    # Filtro per fascia di prezzo
    if price_range_filter:
        price_docs = set(data[data['priceRange'].isin(price_range_filter)].index)
        matching_docs.intersection_update(price_docs)

    # Filtro per regione
    if region_filter:
        region_docs = set(data[data['country'].str.contains('|'.join(region_filter), case=False, na=False)].index)
        matching_docs.intersection_update(region_docs)

    # Filtro per carte di credito accettate
    if credit_card_filter:
        credit_docs = set(data[data['creditCards'].apply(lambda cards: any(card in cards for card in credit_card_filter))].index)
        matching_docs.intersection_update(credit_docs)

    # Filtro per servizi e strutture
    if facilities_filter:
        facilities_docs = set(data[data['facilitiesServices'].apply(lambda facilities: all(facility in facilities for facility in facilities_filter))].index)
        matching_docs.intersection_update(facilities_docs)

    # Restituzione dei ristoranti filtrati
    filtered_restaurants = data.loc[list(matching_docs), ['restaurantName', 'address', 'city', 'priceRange', 'cuisineType', 'website']]
    return filtered_restaurants


# Interazione con l'utente per i nuovi filtri
print("Leave a filter blank if you don't want to apply it.\n")

name_filter = input("Enter restaurant name (or leave blank): ").strip()
city_filter = input("Enter city (or leave blank): ").strip()
cuisine_filter = input("Enter cuisine type (or leave blank): ").strip()

# Filtro per fascia di prezzo
price_range_filter = input("Enter price range (comma-separated, e.g., €€, €€€) or leave blank: ").split(',')
price_range_filter = [p.strip() for p in price_range_filter if p.strip()]  # Rimuovi vuoti

# Filtro per regione
region_filter = input("Enter Italian regions (comma-separated) or leave blank: ").split(',')
region_filter = [r.strip() for r in region_filter if r.strip()]  # Rimuovi vuoti

# Filtro per carte di credito
credit_card_filter = input("Enter accepted credit card types (comma-separated, e.g., Visa, MasterCard) or leave blank: ").split(',')
credit_card_filter = [c.strip() for c in credit_card_filter if c.strip()]  # Rimuovi vuoti

# Filtro per servizi
facilities_filter = input("Enter required facilities/services (comma-separated, e.g., Wi-Fi, Parking) or leave blank: ").split(',')
facilities_filter = [f.strip() for f in facilities_filter if f.strip()]  # Rimuovi vuoti

# Esegui la ricerca avanzata con tutti i filtri
filtered_results = advanced_search_with_filters_enhanced(
    df,
    name_filter=name_filter,
    city_filter=city_filter,
    cuisine_filter=cuisine_filter,
    price_range_filter=price_range_filter if price_range_filter else None,
    region_filter=region_filter if region_filter else None,
    credit_card_filter=credit_card_filter if credit_card_filter else None,
    facilities_filter=facilities_filter if facilities_filter else None
)

# Visualizzazione dei risultati
print(filtered_results)


Leave a filter blank if you don't want to apply it.

     restaurantName         address      city priceRange  \
356  O Me O Il Mare  Via Roma 45/47  Gragnano       €€€€   

                              cuisineType                website  
356  Italian Contemporary, Modern Cuisine  http://omeoilmare.com  


## **Algorithmic Question (AQ)**

In [ ]:
def collect_packages(num_tests, test_cases):
    outcomes = []

    for case_index in range(num_tests):
        num_packages, package_coords = test_cases[case_index]

        # Sort packages by coordinates (x, y) to ensure the smallest lexicographical path
        package_coords.sort()

        path_steps = []
        reachable = True
        curr_x, curr_y = 0, 0  # Start at (0, 0)

        for target_x, target_y in package_coords:
            # Check if the package is reachable from the current position
            if target_x < curr_x or target_y < curr_y:
                # If any package requires moving left or down, mark as unreachable
                reachable = False
                break

            # Append necessary moves to reach the target package
            path_steps.append('R' * (target_x - curr_x))  # Move right
            path_steps.append('U' * (target_y - curr_y))  # Move up

            # Update the current position to the target package's coordinates
            curr_x, curr_y = target_x, target_y

        if reachable:
            # If all packages are reachable, append "YES" and the path
            outcomes.append("YES\n" + ''.join(path_steps))
        else:
            # If any package is unreachable, append "NO"
            outcomes.append("NO")

    return outcomes

# Sample input data
num_tests = 3
test_cases = [
    (5, [(1, 3), (1, 2), (3, 3), (5, 5), (4, 3)]),
    (2, [(1, 0), (0, 1)]),
    (1, [(4, 3)])
]

# Execute function and print results
results = collect_packages(num_tests, test_cases)
for result in results:
    print(result)


YES
RUUURRRRUU
NO
YES
RRRRUUU


### **Pseudocode for the Algorithm**

Given a list of packages located on a grid, where each package is represented by coordinates \((x_i, y_i)\), and a robot starting at \((0, 0)\) that can only move right (`R`) and up (`U`):

1. **Input**:
   - \( t \): the number of test cases.
   - For each test case:
      - \( n \): the number of packages.
      - A list of \( n \) packages with coordinates \((x_i, y_i)\).

2. **Algorithm**:
Start

    Function collect_packages(t, test_cases):
        Results = []   // List to store results for each test case
        
        For each test_case in test_cases:
            (n, packages) = test_case     // Extract number of packages and their coordinates
            Sort packages in ascending order by x, then by y    // Lexicographical sorting

            current_x = 0   // Initial robot position (0, 0)
            current_y = 0
            path = ""    // String to build the path
            possible = True

            For each (x, y) in packages:
                If x < current_x or y < current_y:
                    possible = False   // If the package is unreachable (backward movement)
                    Break the loop

                // Add the necessary moves to reach the package (x, y)
                path = path + "R" * (x - current_x)  // Add right movements
                path = path + "U" * (y - current_y)  // Add upward movements

                // Update the robot's position
                current_x = x
                current_y = y

            If possible:
                Append "YES" and path to Results
            Else:
                Append "NO" to Results

        Return Results   // Return results for all test cases

    // Input reading
    t = input()   // Number of test cases
    test_cases = []   // List to store test cases

    For i = 1 to t:
        n = input()   // Number of packages
        packages = []   // List to store package coordinates

        For j = 1 to n:
            x, y = input()   // Package coordinates
            packages.append((x, y))   // Add the package to the list

        Append (n, packages) to test_cases   // Add the test case to the list

    // Call the function and print the results
    results = collect_packages(t, test_cases)
    For each result in results:
        Print result

End


### **Proof of Correctness**

1. **Sorting Ensures Lexicographical Order**:
   - Sorting the packages by $x$ and then $y$-coordinates ensures that we attempt to collect packages in the lexicographically smallest way.
   - If there is an accessible path for all packages after sorting, this path will be the smallest lexicographical path because it minimizes right and upward movements in order.

2. **Reachability Check**:
   - By moving only right or up from each position, we guarantee that any unreachable package (one that would require moving left or down) will be detected and skipped. The algorithm returns "NO" in such cases.

3. **Path Construction**:
   - For each reachable package, the algorithm appends the correct number of `R` and `U` moves, ensuring that the robot reaches each package in the required order.

4. **Conclusion**:
   - The algorithm is correct because it verifies reachability and constructs the smallest lexicographical path if possible.

### **Time Complexity Analysis**

1. **Sorting the Packages**:
   - Sorting the list of $n$ packages takes $O (\frac{n}{log(n)})$ time.

2. **Constructing the Path**:
   - The path is constructed by iterating over each package, which takes $O(n)$ time. For each package, we calculate the difference in coordinates and append the required moves.

3. **Overall Complexity**:
   - The sorting step, $O (\frac{n}{log(n)})$, is the most time-consuming operation in the algorithm, making the overall complexity: $O (\frac{n}{log(n)})$

### **Verification with a Language Model's Analysis**

If you asked a language model to evaluate this code, it would likely arrive at the same time complexity, $O (\frac{n}{log(n)})$ because sorting is the dominant step in this approach. If there were any discrepancies, they would likely stem from misunderstanding the nature of appending moves as an $O(n)$ operation. However, given that sorting $n$ packages is indeed $O (\frac{n}{log(n)})$ and dominates the time complexity, our analysis is accurate.

### **Extending the Problem: Robot Can Move Left or Down**

With the new rule allowing movement in all directions (right, left, up, down), we consider a **greedy algorithm** where the robot collects the closest package from its current location.

#### **Is the Greedy Algorithm Optimal?**

1. **Greedy Approach**:
   - At each step, the robot moves to the closest package, defined by the Euclidean distance or Manhattan distance from the current position.
   - The robot continues selecting the nearest uncollected package until all packages are collected.

2. **Counterexample to Greedy Optimality**:
   - The greedy approach does not guarantee the minimum path length for the following reason:
   - **Example**:
     - Suppose there are packages at $(0, 1)$, $(1, 0)$, and $(2, 2)$, and the robot starts at $(0, 0)$.
     - The greedy approach would choose either $(0, 1)$ or $(1, 0)$ first, then the remaining one, and finally $(2, 2)$.
     - However, the optimal path would be to go directly to $(2, 2)$ first, then collect $(0, 1)$ and $(1, 0)$, which would result in a shorter total path.

3. **Conclusion**:
   - The greedy algorithm is **not optimal** for minimizing the total distance. While it may be intuitive and yield a quick solution, it does not guarantee the shortest path because the closest package does not necessarily contribute to the globally shortest route for all packages.

